# Inserimento manuale dei dati nel database
### Matter Of Design

In [40]:
pip install mysql-connector-python boto3


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [41]:
# Librerie necessarie
import mysql.connector
import boto3
from tkinter import *
from tkinter import Tk, Label, Entry, Button, messagebox

## Inserimento dati

In [48]:
# Funzione per connettersi al database
def connect_to_db():
    try:
        connection = mysql.connector.connect(
            host='matterofdesign.cboqqic8cvp0.eu-west-2.rds.amazonaws.com',
            user='Massaro',
            password='4PGZUfbq^$pbngQ',
            database='MatterOfDesign'
        )
        if connection.is_connected():
            print("Connected to MySQL database")
            return connection
    except mysql.connector.Error as e:
        print(f"Error: {e}")
        return None

# Funzione per recuperare le tipologie dal database
def fetch_tipologie():
    cursor = db_connection.cursor()
    query = "SELECT tipologia_id, nome FROM Tipologia"
    cursor.execute(query)
    return cursor.fetchall()

# Funzione per caricare un file su Amazon S3
def upload_to_s3(file_path, bucket_name, s3_key):
    s3 = boto3.client('s3')
    try:
        s3.upload_file(file_path, bucket_name, s3_key)
        return f"https://{bucket_name}.s3.amazonaws.com/{s3_key}"
    except Exception as e:
        print(f"Error uploading file: {e}")
        return None

# Funzione per inserire i dati nel database
def insert_data(codice_articolo, brand, finitura, posa, collezione, dimensione, 
                prezzo_unitario, prezzo_mq, colore, spessore, tipologia_id, 
                documento_path, immagine_path, nota, peso, categoria):
    cursor = db_connection.cursor()
    
    # Caricamento del documento e immagine su S3 (se forniti)
    documento_url = upload_to_s3(documento_path, 'matterofdesign-files', f'documenti/{codice_articolo}_doc') if documento_path else None
    immagine_url = upload_to_s3(immagine_path, 'matterofdesign-files', f'immagini/{codice_articolo}_img') if immagine_path else None

    query = """
    INSERT INTO Prodotto (codice_articolo, brand, finitura, posa, collezione, 
    dimensione, prezzo_unitario, prezzo_mq, colore, spessore, 
    tipologia_id, documento, immagine, nota, peso, categoria) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    values = (codice_articolo, brand, finitura, 
              posa if posa else None,
              collezione, dimensione, 
              prezzo_unitario if prezzo_unitario is not None else None, 
              prezzo_mq if prezzo_mq is not None else None, 
              colore, 
              spessore if spessore is not None else None, 
              tipologia_id, 
              documento_url, immagine_url, 
              nota if nota else None, 
              peso if peso is not None else None,
              categoria)
    cursor.execute(query, values)
    db_connection.commit()

# Funzione per raccogliere i dati dall'interfaccia e inserirli
def submit_data():
    try:
        codice_articolo = codice_articolo_entry.get()
        brand = brand_entry.get()
        finitura = finitura_entry.get()
        posa = posa_entry.get()
        collezione = collezione_entry.get()
        dimensione = dimensione_entry.get()
        prezzo_unitario = prezzo_unitario_entry.get()
        prezzo_mq = prezzo_mq_entry.get()
        prezzo_unitario = float(prezzo_unitario) if prezzo_unitario else None
        prezzo_mq = float(prezzo_mq) if prezzo_mq else None
        colore = colore_entry.get()
        spessore = spessore_entry.get()
        spessore = float(spessore) if spessore else None
        tipologia_nome = tipologia_var.get()  # Ottieni il nome selezionato dal menu a tendina
        tipologia_id = tipologia_options[tipologia_nome]  # Ottieni l'ID corrispondente
        documento_path = documento_entry.get()  # Percorso del file documento
        immagine_path = immagine_entry.get()    # Percorso del file immagine
        nota = nota_entry.get()
        peso = peso_entry.get()
        peso = float(peso) if peso else None
        categoria = categoria_entry.get()  # Nuovo campo categoria

        insert_data(codice_articolo, brand, finitura, posa, collezione, dimensione, 
                    prezzo_unitario, prezzo_mq, colore, spessore, tipologia_id, 
                    documento_path, immagine_path, nota, peso, categoria)
        messagebox.showinfo("Success", "Dati inseriti correttamente!")
    except Exception as e:
        messagebox.showerror("Error", f"Errore nell'inserimento dei dati: {e}")

# Connessione al database
db_connection = connect_to_db()

# Creazione dell'interfaccia grafica
root = Tk()
root.title("Inserimento Dati Prodotto")

# Creazione dei campi di input
Label(root, text="Codice Articolo").grid(row=0)
Label(root, text="Brand").grid(row=1)
Label(root, text="Finitura").grid(row=2)
Label(root, text="Posa").grid(row=3)
Label(root, text="Collezione").grid(row=4)
Label(root, text="Dimensione").grid(row=5)
Label(root, text="Prezzo Unitario").grid(row=6)
Label(root, text="Prezzo per MQ").grid(row=7)
Label(root, text="Colore").grid(row=8)
Label(root, text="Spessore").grid(row=9)
Label(root, text="Tipologia").grid(row=10)
Label(root, text="Percorso Documento").grid(row=11)
Label(root, text="Percorso Immagine").grid(row=12)
Label(root, text="Nota").grid(row=13)
Label(root, text="Peso").grid(row=14)
Label(root, text="Categoria").grid(row=15)

# Creazione degli Entry per ogni campo
codice_articolo_entry = Entry(root)
brand_entry = Entry(root)
finitura_entry = Entry(root)
posa_entry = Entry(root)
collezione_entry = Entry(root)
dimensione_entry = Entry(root)
prezzo_unitario_entry = Entry(root)
prezzo_mq_entry = Entry(root)
colore_entry = Entry(root)
spessore_entry = Entry(root)
documento_entry = Entry(root)
immagine_entry = Entry(root)
nota_entry = Entry(root)
peso_entry = Entry(root)
categoria_entry = Entry(root)

# Posizionamento degli Entry nella finestra
codice_articolo_entry.grid(row=0, column=1)
brand_entry.grid(row=1, column=1)
finitura_entry.grid(row=2, column=1)
posa_entry.grid(row=3, column=1)
collezione_entry.grid(row=4, column=1)
dimensione_entry.grid(row=5, column=1)
prezzo_unitario_entry.grid(row=6, column=1)
prezzo_mq_entry.grid(row=7, column=1)
colore_entry.grid(row=8, column=1)
spessore_entry.grid(row=9, column=1)
documento_entry.grid(row=11, column=1)
immagine_entry.grid(row=12, column=1)
nota_entry.grid(row=13, column=1)
peso_entry.grid(row=14, column=1)
categoria_entry.grid(row=15, column=1)

# Menu a tendina per Tipologia
tipologie = fetch_tipologie()  # Recupera le tipologie dal database
tipologia_options = {nome: tipologia_id for tipologia_id, nome in tipologie}  # Crea un dizionario {nome: id}
tipologia_var = StringVar(root)
tipologia_var.set(list(tipologia_options.keys())[0])  # Imposta il valore iniziale (primo elemento)
tipologia_menu = OptionMenu(root, tipologia_var, *tipologia_options.keys())
tipologia_menu.grid(row=10, column=1)

# Pulsante per inviare i dati
Button(root, text="Inserisci", command=submit_data).grid(row=16, column=1)

# Avvio dell'interfaccia
root.mainloop()


Connected to MySQL database


## Correzione dati

In [43]:
# Funzione per connettersi al database
def connect_to_db():
    try:
        connection = mysql.connector.connect(
            host='matterofdesign.cboqqic8cvp0.eu-west-2.rds.amazonaws.com',
            user='Massaro',
            password='4PGZUfbq^$pbngQ',
            database='MatterOfDesign'
        )
        if connection.is_connected():
            print("Connected to MySQL database")
            return connection
    except mysql.connector.Error as e:
        print(f"Error: {e}")
        return None

# Funzione per recuperare le tipologie dal database
def fetch_tipologie():
    cursor = db_connection.cursor()
    query = "SELECT tipologia_id, nome FROM Tipologia"
    cursor.execute(query)
    return cursor.fetchall()

# Funzione per aggiornare i dati nel database
def update_data(codice_articolo, **kwargs):
    cursor = db_connection.cursor()
    fields = ", ".join(f"{key} = %s" for key in kwargs.keys() if kwargs[key] is not None)
    query = f"UPDATE Prodotto SET {fields} WHERE codice_articolo = %s"
    values = tuple(kwargs[key] for key in kwargs.keys() if kwargs[key] is not None) + (codice_articolo,)

    cursor.execute(query, values)
    db_connection.commit()

# Funzione per ripulire i campi
def clear_fields():
    brand_entry.delete(0, END)
    finitura_entry.delete(0, END)
    posa_entry.delete(0, END)
    collezione_entry.delete(0, END)
    dimensione_entry.delete(0, END)
    prezzo_unitario_entry.delete(0, END)
    prezzo_mq_entry.delete(0, END)
    colore_entry.delete(0, END)
    spessore_entry.delete(0, END)
    tipologia_var.set(list(tipologia_options.keys())[0])  # Reset del menu a tendina
    nota_entry.delete(0, END)
    peso_entry.delete(0, END)
    categoria_entry.delete(0, END)

# Funzione per caricare i dati esistenti per un prodotto specifico
def load_product_data():
    clear_fields()  # Ripulisce tutti i campi prima di caricare i nuovi dati
    codice_articolo = codice_articolo_entry.get()
    cursor = db_connection.cursor()
    cursor.execute("SELECT * FROM Prodotto WHERE codice_articolo = %s", (codice_articolo,))
    result = cursor.fetchone()

    if result:
        # Carica i valori nei campi di input
        brand_entry.insert(0, result[1] if result[1] else "")
        finitura_entry.insert(0, result[2] if result[2] else "")
        posa_entry.insert(0, result[3] if result[3] else "")
        collezione_entry.insert(0, result[4] if result[4] else "")
        dimensione_entry.insert(0, result[5] if result[5] else "")
        prezzo_unitario_entry.insert(0, result[6] if result[6] else "")
        prezzo_mq_entry.insert(0, result[7] if result[7] else "")
        colore_entry.insert(0, result[8] if result[8] else "")
        spessore_entry.insert(0, result[9] if result[9] else "")
        tipologia_nome = next((nome for nome, id in tipologia_options.items() if id == result[10]), None)
        tipologia_var.set(tipologia_nome if tipologia_nome else list(tipologia_options.keys())[0])
        nota_entry.insert(0, result[13] if result[13] else "")
        peso_entry.insert(0, result[14] if result[14] else "")
        categoria_entry.insert(0, result[15] if result[15] else "")  # Campo Categoria
    else:
        messagebox.showerror("Errore", "Prodotto non trovato!")

def submit_update():
    try:
        codice_articolo = codice_articolo_entry.get()
        tipologia_nome = tipologia_var.get()  # Ottieni il nome selezionato dal menu a tendina
        tipologia_id = tipologia_options[tipologia_nome]  # Ottieni l'ID corrispondente
        update_data(
            codice_articolo,
            brand=brand_entry.get() if brand_entry.get() else None,
            finitura=finitura_entry.get() if finitura_entry.get() else None,
            posa=posa_entry.get() if posa_entry.get() else None,
            collezione=collezione_entry.get() if collezione_entry.get() else None,
            dimensione=dimensione_entry.get() if dimensione_entry.get() else None,
            prezzo_unitario=float(prezzo_unitario_entry.get()) if prezzo_unitario_entry.get() else None,
            prezzo_mq=float(prezzo_mq_entry.get()) if prezzo_mq_entry.get() else None,
            colore=colore_entry.get() if colore_entry.get() else None,
            spessore=float(spessore_entry.get()) if spessore_entry.get() else None,
            tipologia_id=tipologia_id,  # Usa l'ID selezionato dal menu a tendina
            nota=nota_entry.get() if nota_entry.get() else None,
            peso=float(peso_entry.get()) if peso_entry.get() else None,
            categoria=categoria_entry.get() if categoria_entry.get() else None  # Campo Categoria
        )
        messagebox.showinfo("Successo", "Dati aggiornati correttamente!")
        clear_fields()  # Ripulisce i campi dopo l'aggiornamento
    except Exception as e:
        messagebox.showerror("Errore", f"Errore nell'aggiornamento dei dati: {e}")

# Connessione al database
db_connection = connect_to_db()

# Recupera le tipologie dal database
tipologie = fetch_tipologie()
tipologia_options = {nome: tipologia_id for tipologia_id, nome in tipologie}  # Crea un dizionario {nome: id}

# Creazione dell'interfaccia grafica per l'aggiornamento
root = Tk()
root.title("Modifica Dati Prodotto")

# Campi per il codice articolo e caricamento dati
Label(root, text="Codice Articolo").grid(row=0, column=0)
codice_articolo_entry = Entry(root)
codice_articolo_entry.grid(row=0, column=1)

Button(root, text="Carica Dati", command=load_product_data).grid(row=0, column=2)

# Campi di input per gli altri dati
Label(root, text="Brand").grid(row=1, column=0)
Label(root, text="Finitura").grid(row=2, column=0)
Label(root, text="Posa").grid(row=3, column=0)
Label(root, text="Collezione").grid(row=4, column=0)
Label(root, text="Dimensione").grid(row=5, column=0)
Label(root, text="Prezzo Unitario").grid(row=6, column=0)
Label(root, text="Prezzo per MQ").grid(row=7, column=0)
Label(root, text="Colore").grid(row=8, column=0)
Label(root, text="Spessore").grid(row=9, column=0)
Label(root, text="Tipologia").grid(row=10, column=0)
Label(root, text="Nota").grid(row=11, column=0)
Label(root, text="Peso").grid(row=12, column=0)
Label(root, text="Categoria").grid(row=13, column=0)  # Campo Categoria

# Creazione degli entry per i dati
brand_entry = Entry(root)
finitura_entry = Entry(root)
posa_entry = Entry(root)
collezione_entry = Entry(root)
dimensione_entry = Entry(root)
prezzo_unitario_entry = Entry(root)
prezzo_mq_entry = Entry(root)
colore_entry = Entry(root)
spessore_entry = Entry(root)
nota_entry = Entry(root)
peso_entry = Entry(root)
categoria_entry = Entry(root)  # Campo Categoria

brand_entry.grid(row=1, column=1)
finitura_entry.grid(row=2, column=1)
posa_entry.grid(row=3, column=1)
collezione_entry.grid(row=4, column=1)
dimensione_entry.grid(row=5, column=1)
prezzo_unitario_entry.grid(row=6, column=1)
prezzo_mq_entry.grid(row=7, column=1)
colore_entry.grid(row=8, column=1)
spessore_entry.grid(row=9, column=1)
nota_entry.grid(row=11, column=1)
peso_entry.grid(row=12, column=1)
categoria_entry.grid(row=13, column=1)

# Menu a tendina per Tipologia
tipologia_var = StringVar(root)
tipologia_var.set(list(tipologia_options.keys())[0])  # Imposta il valore iniziale
tipologia_menu = OptionMenu(root, tipologia_var, *tipologia_options.keys())
tipologia_menu.grid(row=10, column=1)

# Pulsante per inviare i dati aggiornati
Button(root, text="Aggiorna", command=submit_update).grid(row=14, column=1)

# Avvio dell'interfaccia
root.mainloop()

Connected to MySQL database


## Eliminazione dati

In [47]:
# Funzione per connettersi al database
def connect_to_db():
    try:
        connection = mysql.connector.connect(
            host='matterofdesign.cboqqic8cvp0.eu-west-2.rds.amazonaws.com',
            user='Massaro',
            password='4PGZUfbq^$pbngQ',
            database='MatterOfDesign'
        )
        if connection.is_connected():
            print("Connected to MySQL database")
            return connection
    except mysql.connector.Error as e:
        print(f"Error: {e}")
        return None

# Funzione per eliminare un prodotto dal database
def delete_product(codice_articolo):
    try:
        cursor = db_connection.cursor()
        # Controlla se il prodotto esiste prima di eliminarlo
        cursor.execute("SELECT * FROM Prodotto WHERE codice_articolo = %s", (codice_articolo,))
        result = cursor.fetchone()
        if result:
            # Conferma eliminazione
            response = messagebox.askyesno("Conferma", f"Sei sicuro di voler eliminare il prodotto con codice articolo {codice_articolo}?")
            if response:
                cursor.execute("DELETE FROM Prodotto WHERE codice_articolo = %s", (codice_articolo,))
                db_connection.commit()
                messagebox.showinfo("Successo", "Prodotto eliminato correttamente!")
        else:
            messagebox.showerror("Errore", f"Prodotto con codice articolo {codice_articolo} non trovato.")
    except Exception as e:
        messagebox.showerror("Errore", f"Errore durante l'eliminazione del prodotto: {e}")

# Funzione per gestire l'eliminazione tramite l'interfaccia
def submit_deletion():
    codice_articolo = codice_articolo_entry.get()  # Ottiene il codice articolo dall'interfaccia
    if not codice_articolo:
        messagebox.showerror("Errore", "Inserire un codice articolo valido.")
        return
    delete_product(codice_articolo)
    codice_articolo_entry.delete(0, END)  # Ripulisce il campo di input

# Connessione al database
db_connection = connect_to_db()

# Creazione dell'interfaccia grafica
root = Tk()
root.title("Elimina Prodotto")

# Creazione dei campi di input
Label(root, text="Codice Articolo da Eliminare").grid(row=0, column=0, padx=10, pady=10)
codice_articolo_entry = Entry(root)
codice_articolo_entry.grid(row=0, column=1, padx=10, pady=10)

# Pulsante per confermare l'eliminazione
Button(root, text="Elimina", command=submit_deletion).grid(row=1, column=1, pady=10)

# Avvio dell'interfaccia
root.mainloop()

Connected to MySQL database


## Verifica dati

In [46]:
# Funzione per connettersi al database
def connect_to_db():
    try:
        connection = mysql.connector.connect(
            host='matterofdesign.cboqqic8cvp0.eu-west-2.rds.amazonaws.com',
            user='Massaro',
            password='4PGZUfbq^$pbngQ',
            database='MatterOfDesign'
        )
        if connection.is_connected():
            print("Connected to MySQL database")
            return connection
    except mysql.connector.Error as e:
        print(f"Error: {e}")
        return None

def fetch_data():
    connection = connect_to_db()
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM Prodotto")
    results = cursor.fetchall()
    for row in results:
        print(row)

# Esegui la funzione per vedere i dati
fetch_data()

Connected to MySQL database
('33100101', 'Alice Ceramica', 'Lucido', 'Sospesa', 'Nur', '55x45', Decimal('562.00'), None, 'Bianco', None, 11, None, None, 'Vaso scarico a p trasformabile a s, fornito con riduttore di flusso', None, 'Vaso')
('33100101.08', 'Alice Ceramica', 'Opaco', 'Sospesa', 'Nur', '55x45', Decimal('843.00'), None, 'Nero', None, 11, None, None, 'Vaso scarico a p trasformabile a s, fornito con riduttore di flusso', None, 'Vaso')
('33100101.09', 'Alice Ceramica', 'Opaco', 'Sospesa', 'Nur', '55x45', Decimal('843.00'), None, 'Bianco', None, 11, None, None, 'Vaso scarico a p trasformabile a s, fornito con riduttore di flusso', None, 'Vaso')
('33100101.12', 'Alice Ceramica', 'Lucido', 'Sospesa', 'Nur', '55x45', Decimal('843.00'), None, 'Platino', None, 11, None, None, 'Vaso scarico a p trasformabile a s, fornito con riduttore di flusso', None, 'Vaso')
('33100101.14', 'Alice Ceramica', 'Lucido', 'Sospesa', 'Nur', '55x45', Decimal('843.00'), None, 'Oro', None, 11, None, None, '